In [63]:
pip install bs4


In [64]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [65]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [66]:
from bs4 import BeautifulSoup
import requests as req
import pandas as pd
import os
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [67]:
df = pd.read_excel('C:/blackcofferassignment/Input.xlsx')
for index in range(len(df)):
    url = df['URL'][index]
    urlid = df['URL_ID'][index]
    try:
        res = req.get(url)
    except:
        print(f"Error in accessing the data in url:\n {htmltext.title.text.strip()}")
    try:
        htmltext = BeautifulSoup(res.text,'html.parser')
    except :
        print("Error: No text was found in the article:",e)
        print(htmltext.title.text.strip())
    article = ""
    try:
        for p in htmltext.find_all('p'):
             article += p.get_text()

        title = htmltext.find('h1').get_text()
    except:
        print("Error in parsing of ",urlid)
    
    file_name = "C:/blackcofferassignment/content/" + str(urlid) + '.txt'
    with open(file_name,'w',encoding = 'utf-8') as file:
        file.write(title+'\n'+article)





Error in parsing of  blackassign0036
Error in parsing of  blackassign0049


In [68]:
# Load stopwords
stop = set()
stopwords_dir = "C:/blackcofferassignment/StopWords"
for file in os.listdir(stopwords_dir):
    path = os.path.join(stopwords_dir, file)
    with open(path, 'r', encoding='ISO-8859-1') as f:
        stop.update(set(line.strip().lower() for line in f))

# Load content from text files
content_dir = "C:/blackcofferassignment/content/"
content = []

for text_file in os.listdir(content_dir):
    with open(os.path.join(content_dir, text_file), 'r', encoding='ISO-8859-1') as f:
        text = f.read()
        words = nltk.tokenize.word_tokenize(text)
        clean_text = [word for word in words if word.lower() not in stop]
        content.append(clean_text)

# Load positive and negative words
pos = set()
neg = set()

with open("C:/blackcofferassignment/MasterDictionary/positive-words.txt", 'r', encoding='ISO-8859-1') as f:
    pos.update(f.read().splitlines())

with open("C:/blackcofferassignment/MasterDictionary/negative-words.txt", 'r', encoding='ISO-8859-1') as f:
    neg.update(f.read().splitlines())

# Analyze sentiment scores
pos_words = []
neg_words = []
pos_score = []
neg_score = []
polarity_score = []
subjectivity_score = []

for i in range(len(content)):
    pos_words.append([word for word in content[i] if word.lower() in pos])
    neg_words.append([word for word in content[i] if word.lower() in neg])
    pos_score.append(len(pos_words[i]))
    neg_score.append(len(neg_words[i]))
    polarity_score.append((pos_score[i] - neg_score[i]) / (pos_score[i] + neg_score[i] + 0.000001))
    subjectivity_score.append((pos_score[i] + neg_score[i]) / (len(content[i]) + 0.000001))


In [70]:
import re
avg_sentence_list = []
complex_words_percent_list = []
fog_index_list = []
complex_words_len_list = []
avg_syllable_list = []

def function(file):
    with open(os.path.join(content_dir,file),'r',encoding= 'ISO-8859-1') as f:
        text = f.read()
        text = re.sub(r'[^\w\s.]','',text)
        sentences = nltk.tokenize.sent_tokenize(text)
        num_sen = len(sentences)

        words = [ word for word in text.split() if word.lower() not in stop]
        num_word = len(words)

        avg_sentence = num_word / num_sen

        complex_words = []
        for word in words:
            vowels = 'aeiou'
            syllable_count = sum(1 for letter in word if letter.lower() in vowels)
            if(syllable_count > 2):
                complex_words.append(word)

        syllable_count = 0
        syllable_words = []
        for word in words:
            if(word.endswith("es") or word.endswith("ed")):
                word = word[:-2]
            vowels = "aeiou"
            count = sum(1 for letter in word if letter.lower() in vowels)
            if count>=1:
                syllable_words.append(word)
                syllable_count += count 
            avg_syllable = syllable_count/len(syllable_words)
            complex_words_percent = len(complex_words)/num_word
            fog_index = 0.4 * (avg_sentence + complex_words_percent)

            return avg_sentence,complex_words_percent,fog_index,len(complex_words),avg_syllable
for file in os.listdir(content_dir):
    avg_sentence,complex_words_percent,fog_index,complex_words_len,avg_syllable = function(file)
    avg_sentence_list.append(avg_sentence)
    complex_words_percent_list.append(complex_words_percent)
    fog_index_list.append(fog_index)
    complex_words_len_list.append(complex_words_len)
    avg_syllable_list.append(avg_syllable)
            




In [71]:
def cleaned_words(file):
  with open(os.path.join(content_dir,file), 'r',encoding='ISO-8859-1') as f:
    text = f.read()
    text = re.sub(r'[^\w\s]', '' , text)
    words = [word  for word in text.split() if word.lower() not in stop
            ]
    length = sum(len(word) for word in words)
    average_word_length = length / len(words)
  return len(words),average_word_length

word_count = []
average_word_length = []
for file in os.listdir(content_dir):
  x,y = cleaned_words(file)
  word_count.append(x)
  average_word_length.append(y)

In [72]:
def count_personal_pronouns(file):
  with open(os.path.join(content_dir,file), 'r',encoding='ISO-8859-1') as f:
    text = f.read()
    personal_pronouns = ["I", "we", "my", "ours", "us"]
    count = 0
    for pronoun in personal_pronouns:
      count += len(re.findall(r"\b" + pronoun + r"\b", text)) # \b is used to match word boundaries
  return count

pp_count = []
for file in os.listdir(content_dir):
  x = count_personal_pronouns(file)
  pp_count.append(x)

In [74]:
answer = pd.read_excel("C:/blackcofferassignment/Output Data Structure.xlsx")

#answer.drop([36,49],axis = 0,inplace = True)

variables= [pos_score, neg_score,polarity_score,subjectivity_score,average_word_length,complex_words_percent_list,fog_index_list,avg_sentence_list,complex_words_len_list,word_count,avg_syllable_list,pp_count,average_word_length]

for i,var in enumerate(variables):
    answer.iloc[:,i+2] = var

answer.to_excel('Output Data Structure.xlsx')

In [75]:
check = pd.DataFrame(variables)
check

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,6.000000,54.000000,40.000000,40.000000,21.000000,88.000000,25.000000,32.000000,38.000000,60.000000,...,25.000000,19.000000,7.000000,32.000000,10.000000,29.000000,25.000000,5.000000,14.000000,32.000000
1,1.000000,29.000000,23.000000,71.000000,8.000000,27.000000,34.000000,9.000000,46.000000,66.000000,...,27.000000,37.000000,3.000000,43.000000,23.000000,51.000000,34.000000,0.000000,2.000000,51.000000
2,0.714286,0.301205,0.269841,-0.279279,0.448276,0.530435,-0.152542,0.560976,-0.095238,-0.047619,...,-0.038462,-0.321429,0.400000,-0.146667,-0.393939,-0.275000,-0.152542,1.000000,0.750000,-0.228916
3,0.020649,0.083000,0.070234,0.126712,0.052347,0.084559,0.085014,0.055256,0.098361,0.127016,...,0.077037,0.064740,0.039683,0.097276,0.059891,0.102302,0.090352,0.019157,0.036117,0.124251
4,7.448669,7.888325,8.414127,8.352941,8.028698,8.364203,7.697133,8.302048,8.360058,8.301370,...,7.469565,7.751055,7.390863,7.097792,7.317016,7.548485,7.217726,7.725888,7.263006,7.612022
5,0.550186,0.630653,0.659280,0.641834,0.583700,0.654255,0.567616,0.657046,0.660844,0.632880,...,0.560554,0.527933,0.535354,0.496894,0.511416,0.539970,0.537285,0.583756,0.501408,0.552536
6,4.898335,5.306230,6.408393,7.415708,6.959406,7.539121,5.010025,5.384557,7.134338,5.832462,...,8.196635,12.144507,11.528427,5.923202,10.510449,6.109321,7.963062,19.933503,6.374476,11.842067
7,11.695652,12.634921,15.361702,17.897436,16.814815,18.193548,11.957447,12.804348,17.175000,13.948276,...,19.931034,29.833333,28.285714,14.311111,25.764706,14.733333,19.370370,49.250000,15.434783,29.052632
8,148.000000,502.000000,476.000000,448.000000,265.000000,738.000000,319.000000,387.000000,454.000000,512.000000,...,324.000000,378.000000,106.000000,320.000000,224.000000,358.000000,281.000000,115.000000,178.000000,305.000000
9,263.000000,788.000000,722.000000,697.000000,453.000000,1123.000000,558.000000,586.000000,686.000000,803.000000,...,575.000000,711.000000,197.000000,634.000000,429.000000,660.000000,519.000000,197.000000,346.000000,549.000000
